In [1]:
import os, sys
sys.path.append("..")

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

import numpy as np
import torch
import torch.nn as nn
import torchvision
import gc

from src.tools import unfreeze, freeze
from src.tools import load_dataset, get_loader_stats

from copy import deepcopy
import json

from tqdm import tqdm_notebook as tqdm
from IPython.display import clear_output

# This needed to use dataloaders for some datasets
from PIL import PngImagePlugin
LARGE_ENOUGH_NUMBER = 100
PngImagePlugin.MAX_TEXT_CHUNK = LARGE_ENOUGH_NUMBER * (1024**2)

/home/user/conda/lib/python3.7/site-packages/librosa/core/constantq.py:1059: DeprecationWarning: `np.complex` is a deprecated alias for the builtin `complex`. To silence this warning, use `complex` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.complex128` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.complex,


In [2]:
DEVICE_ID = 0

DATASET_LIST = [
#    ('handbag', '../../data/handbag_128.hdf5', 64),
#    ('handbag', '../../data/handbag_128.hdf5', 128),
#    ('shoes', '../../data/shoes_128.hdf5', 64),
#    ('shoes', '../../data/shoes_128.hdf5', 128),
    ('celeba_female', '/home/jovyan/data/dataset/woman.pt', 512),
    ('celeba_male', '/home/jovyan/data/dataset/man.pt', 512),
#    ('aligned_anime_faces', '../../data/aligned_anime_faces', 64),
#    ('aligned_anime_faces', '../../data/aligned_anime_faces', 128),
]

assert torch.cuda.is_available()
torch.cuda.set_device(f'cuda:{DEVICE_ID}')

In [3]:
sampler, test_sampler = load_dataset('celeba_female_tensor', '/home/jovyan/data/dataset/woman.pt', img_size=512, batch_size = 32)

42905


In [4]:
from src.vae import latent_to_pil_from_bath_of_tensor

In [5]:
dataset = []

for batch in tqdm(test_sampler.loader):
    latent = batch[0]
    tensor = latent_to_pil_from_bath_of_tensor(latent)
    dataset.append(tensor)

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


  0%|          | 0/135 [00:00<?, ?it/s]

In [8]:
len(dataset)

135

In [11]:
tensor = torch.vstack(dataset)

In [47]:
tensor.shape

torch.Size([4290, 3, 512, 512])

In [19]:
from torch.utils.data import TensorDataset, DataLoader, Subset
from src.distributions import LoaderSampler

In [48]:
test_sampler = LoaderSampler(DataLoader(TensorDataset(tensor, torch.zeros(tensor.shape[0])), shuffle=False, num_workers=0, batch_size=32), 'cuda')

In [49]:
mu, sigma = get_loader_stats(test_sampler.loader)

In [15]:
stats = {'mu' : mu.tolist(), 'sigma' : sigma.tolist()}

In [16]:
filename = 'celeba_female_tensor_512_test.json'
with open(filename, 'w') as fp:
    json.dump(stats, fp)

In [50]:
t_1 = tensor[:2145]
t_2 = tensor[2145:]

In [52]:
t_2.shape

torch.Size([2145, 3, 512, 512])

In [53]:
t_1 = tensor[:2145]
t_2 = tensor[2145:]


test_sampler_1 = LoaderSampler(DataLoader(TensorDataset(t_1, torch.zeros(t_1.shape[0])), shuffle=False, num_workers=0, batch_size=32), 'cuda')
test_sampler_2 = LoaderSampler(DataLoader(TensorDataset(t_2, torch.zeros(t_1.shape[0])), shuffle=False, num_workers=0, batch_size=32), 'cuda')

In [54]:
mu_1, sigma_1 = get_loader_stats(test_sampler_1.loader)

In [55]:
mu_2, sigma_2 = get_loader_stats(test_sampler_2.loader)

In [56]:
calculate_frechet_distance(mu_1, sigma_1, mu_2, sigma_2)

13.500458550441692

In [17]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import Compose, Resize, Normalize, ToTensor, RandomCrop

In [ ]:
transform = Compose([Resize((512, 512)), ToTensor(), Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
dataset = ImageFolder('/home/jovyan/data/dataset/data', transform=transform)

In [ ]:
idx = list(range(len(dataset)))
test_size = int(len(idx) * 0.1)
train_idx, test_idx = idx[:-test_size], idx[-test_size:]
train_set, test_set = Subset(dataset, train_idx), Subset(dataset, test_idx)

In [ ]:
dataset[0].shape

In [ ]:
train_idx[:test_size]

In [ ]:
test_set

In [ ]:
test_sampler1 = LoaderSampler(test_set)

In [ ]:
for batch in test_sampler1.loader:
    print(batch.shape)

In [23]:
test_sampler1 = LoaderSampler(DataLoader(test_set, shuffle=True, num_workers=0, batch_size=32), 'cuda')

In [33]:
from src.inception import InceptionV3


def get_loader_stats1(loader, batch_size=8, verbose=False):
    dims = 2048
    block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[dims]
    model = InceptionV3([block_idx]).cuda()
    freeze(model)
    
    size = len(loader.dataset)
    pred_arr = []
    
    with torch.no_grad():
        for step, X in enumerate(loader) if not verbose else tqdm(enumerate(loader)):
            for i in range(0, len(X), batch_size):
                start, end = i, min(i + batch_size, len(X))
                batch = ((X[start:end] + 1) / 2).type(torch.FloatTensor).cuda()
                pred_arr.append(model(batch)[0].cpu().data.numpy().reshape(end-start, -1))

    pred_arr = np.vstack(pred_arr)
    mu, sigma = np.mean(pred_arr, axis=0), np.cov(pred_arr, rowvar=False)
    gc.collect(); torch.cuda.empty_cache()
    return mu, sigma

In [34]:
mu1, sigma1 = get_loader_stats1(test_sampler1.loader)

In [36]:
from src.fid_score import calculate_frechet_distance

fid = calculate_frechet_distance(mu, sigma, mu1, sigma1)

In [37]:
fid

32.95988819246031

In [ ]:
for DATASET, DATASET_PATH, IMG_SIZE in tqdm(DATASET_LIST):
    print('Processing {}'.format(DATASET))
    sampler, test_sampler = load_dataset(DATASET, DATASET_PATH, img_size=IMG_SIZE)
    
    
    
    print('Dataset {} loaded'.format(DATASET))

    mu, sigma = get_loader_stats(test_sampler.loader)
    print('Trace of sigma: {}'.format(np.trace(sigma)))
    stats = {'mu' : mu.tolist(), 'sigma' : sigma.tolist()}
    print('Stats computed')

    filename = '{}_{}_test.json'.format(DATASET, IMG_SIZE)
    with open(filename, 'w') as fp:
        json.dump(stats, fp)
    print('States saved to {}'.format(filename))